<a href="https://colab.research.google.com/github/Omanshugupta/Conversational_agent/blob/main/Conversational_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import google.generativeai as genai
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from dotenv import load_dotenv

In [11]:
from google.colab import userdata
api_key=userdata.get('gemini_key')

In [12]:
genai.configure(api_key=api_key)

In [14]:
load_dotenv()
llm = genai.GenerativeModel(model_name="gemini-2.0-flash", generation_config={"max_output_tokens": 1000, "temperature": 0})

In [15]:
store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [34]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
], input_variables=["input", "history"])

In [53]:
from langchain_core.runnables import Runnable
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables import RunnablePassthrough
import google.generativeai as genai

class GeminiStreamRunnable(Runnable):
    def __init__(self, model):
        self.model = model

    def stream(self, input, config=None):
        # Input is now expected to be a list of messages in the format
        # expected by the Gemini API, prepared by a previous step in the chain.
        gemini_messages = input

        # Call the model's streaming method using generate_content with stream=True
        generation_config = config.get("generation_config", {})
        safety_settings = config.get("safety_settings", [])

        for chunk in self.model.generate_content(
            gemini_messages,
            stream=True,
            generation_config=generation_config,
            safety_settings=safety_settings
        ):
             if chunk.candidates:
                 for candidate in chunk.candidates:
                     if candidate.content and candidate.content.parts:
                         for part in candidate.content.parts:
                             if part.text:
                                 yield part.text

    def invoke(self, input, config=None):
        result = ""
        for chunk in self.stream(input, config):
            result += chunk
        return result

class MessageFormattingRunnable(Runnable):
    """A runnable to format messages for the Gemini API."""
    def invoke(self, input_dict, config=None):
        """Formats the input dictionary with 'input' and 'history' into Gemini's message format."""
        history = input_dict.get("history", [])
        current_input = input_dict.get("input")

        gemini_messages = []
        for msg in history:
            if isinstance(msg, HumanMessage):
                gemini_messages.append({'role': 'user', 'parts': [msg.content]})
            elif isinstance(msg, AIMessage):
                gemini_messages.append({'role': 'model', 'parts': [msg.content]})
            elif isinstance(msg, SystemMessage):
                 print(f"Warning: Skipping SystemMessage in chat history.")
                 continue
            else:
                print(f"Warning: Skipping unsupported message type in history: {type(msg)}")
                continue

        if current_input is not None:
             if isinstance(current_input, str):
                 gemini_messages.append({'role': 'user', 'parts': [current_input]})
             elif isinstance(current_input, list):
                  gemini_messages.append({'role': 'user', 'parts': current_input})

        return gemini_messages

# Define the chain:
# 1. Receive the input dictionary with 'input' and 'history' (handled by RunnableWithMessageHistory)
# 2. Format the messages for Gemini using MessageFormattingRunnable
# 3. Pass the formatted messages to the GeminiStreamRunnable
chain = MessageFormattingRunnable() | GeminiStreamRunnable(llm)

In [52]:

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [57]:
session_id = "user_123"


response1 = chain_with_history.invoke(
    {"input": "Hello! How are you?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1)

response2 = chain_with_history.invoke(
    {"input": "What was my previous message?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2)

AI: I'm doing well, thank you for asking! How can I help you today?

AI: Your previous message was: "Hello! How are you?"



In [58]:

print("\nConversation History:")
for message in store[session_id].messages:
    print(f"{message.type}: {message.content}")


Conversation History:
human: Hello! How are you?
ai: I am doing well, thank you for asking! As a large language model, I don't experience emotions or feelings like humans do, but I am functioning optimally and ready to assist you. How can I help you today?

human: What was my previous message?
ai: Your previous message was: "Hello! How are you?"

human: Hello! How are you?
ai: I am doing well, thank you for asking! How can I help you today?

human: What was my previous message?
ai: Your previous message was: "Hello! How are you?"

human: Hello! How are you?
ai: I'm doing well, thank you for asking! How can I help you today?

human: What was my previous message?
ai: Your previous message was: "Hello! How are you?"

